In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math
from sklearn.model_selection import train_test_split,KFold
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn import preprocessing
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
print(os.listdir("../input"))

In [ ]:
# reading the data

train_df= pd.read_csv('../input/train.csv')
test_df= pd.read_csv('../input/test.csv')

# getting the shapes of the datasets
print("Shape of Train :", train_df.shape)
print("Shape of Test :", test_df.shape)

In [ ]:
# lets combine the train and test for data prep
test_df['amount_spent_per_room_night_scaled']=np.nan
train_df['data']='train'
test_df['data']='test'
#test=test[train.columns]
all=pd.concat([train_df,test_df],axis=0)
reservid=test_df['reservation_id']

In [ ]:
from datetime import datetime
all['booking_date'] = all['booking_date'].apply(lambda x: datetime.strptime(x,"%d/%m/%y"))
all['checkin_date'] = all['checkin_date'].apply(lambda x: datetime.strptime(x,"%d/%m/%y"))
all['checkout_date'] = all['checkout_date'].apply(lambda x: datetime.strptime(x,"%d/%m/%y"))


In [ ]:
all['checkin_year']=all['checkin_date'].dt.year
all['bookin_year']=all['booking_date'].dt.year

all['checkin_month']=all['checkin_date'].dt.month
all['bookin_month']=all['booking_date'].dt.month

all['checkin_day']=all['checkin_date'].dt.day
all['bookin_day']=all['booking_date'].dt.day

In [ ]:
#all['member_age_buckets'].value_counts()
all['member_age_buckets'] = all['member_age_buckets'].map({'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9})


In [ ]:
a=pd.DataFrame(all['checkin_date']-all['booking_date'])
all['book_checkin_diff']=a[0].astype(str).str.split(" ", n = 1, expand = True)[0]

b=pd.DataFrame(all['checkout_date']-all['checkin_date'])
all['days_stay']=b[0].astype(str).str.split(" ", n = 1, expand = True)[0]



In [ ]:
for col in ['reservation_id','booking_date','checkin_date','checkout_date','memberid']:
     all.drop([col],axis=1,inplace=True)

In [ ]:
#all.isnull().sum()

In [ ]:
#impute missing values using mode(frequency)

#mode_value=all['season_holidayed_code'].mode()  returned  #2.0
all["season_holidayed_code"].fillna(2.0, inplace = True) 
#mode_value=all['state_code_residence'].mode() returned #8.0
all['state_code_residence'].fillna(8.0, inplace = True) 

In [ ]:
all.dtypes

In [ ]:
for col in ['resort_id','cluster_code','season_holidayed_code','bookin_year']:
    dum=pd.get_dummies(all[col],drop_first=True,prefix='A')
    all=pd.concat([all,dum],axis=1)
    all.drop([col],axis=1,inplace=True)
    
for col in ['channel_code','reservationstatusid_code','checkin_year']:
    dum=pd.get_dummies(all[col],drop_first=True,prefix='B')
    all=pd.concat([all,dum],axis=1)
    all.drop([col],axis=1,inplace=True)
                        
dum=pd.get_dummies(all['main_product_code'],drop_first=True,prefix='C')
all=pd.concat([all,dum],axis=1)
all.drop(['main_product_code'],axis=1,inplace=True)
            
dum=pd.get_dummies(all['resort_region_code'],drop_first=True,prefix='D')
all=pd.concat([all,dum],axis=1)
all.drop(['resort_region_code'],axis=1,inplace=True)
            
dum=pd.get_dummies(all['state_code_residence'],drop_first=True,prefix='E')
all=pd.concat([all,dum],axis=1)
all.drop(['state_code_residence'],axis=1,inplace=True)

dum=pd.get_dummies(all['persontravellingid'],drop_first=True,prefix='F')
all=pd.concat([all,dum],axis=1)
all.drop(['persontravellingid'],axis=1,inplace=True)

dum=pd.get_dummies(all['state_code_resort'],drop_first=True,prefix='G')
all=pd.concat([all,dum],axis=1)
all.drop(['state_code_resort'],axis=1,inplace=True)

In [ ]:
dum=pd.get_dummies(all['checkin_month'],drop_first=True,prefix='I')
all=pd.concat([all,dum],axis=1)
all.drop(['checkin_month'],axis=1,inplace=True)

dum=pd.get_dummies(all['bookin_month'],drop_first=True,prefix='J')
all=pd.concat([all,dum],axis=1)
all.drop(['bookin_month'],axis=1,inplace=True)

dum=pd.get_dummies(all['checkin_day'],drop_first=True,prefix='K')
all=pd.concat([all,dum],axis=1)
all.drop(['checkin_day'],axis=1,inplace=True)

dum=pd.get_dummies(all['bookin_day'],drop_first=True,prefix='L')
all=pd.concat([all,dum],axis=1)
all.drop(['bookin_day'],axis=1,inplace=True)

In [ ]:
all['book_checkin_diff']=all['book_checkin_diff'].astype(int)
all['days_stay']=all['days_stay'].astype(int)

In [ ]:
train=all[all['data']=='train']
del train['data']
test=all[all['data']=='test']
test.drop(['amount_spent_per_room_night_scaled','data'],axis=1,inplace=True)

In [ ]:
#t1,t2 = train_test_split(train,test_size=0.30,random_state=12)

#t1_x=t1.drop(['amount_spent_per_room_night_scaled'],axis=1)
#t1_y=t1['amount_spent_per_room_night_scaled']
#t2_x=t2.drop(['amount_spent_per_room_night_scaled'],axis=1)
#t2_y=t2['amount_spent_per_room_night_scaled']

In [ ]:
#estimator = LGBMRegressor()

#params = {
#    'learning_rate': [0.03,0.05],
#    'boosting_type':['gbdt'],
#   'n_estimators': [630,650],
#    'subsample':[0.75,0.8],
#    'num_leaves':[50,55,52],
#    'colsample_bytree':[0.5,0.57,0.6],
#   'reg_alpha' : [5.5],
#   'reg_lambda' : [4],
#    'objective':['regression']
#   }
#model = RandomizedSearchCV(estimator,param_distributions=params,cv=4,verbose=1,n_iter=20,n_jobs=-1)

#model.fit(t1_x,t1_y)

In [ ]:
#pred=model.predict(t2_x)
#mean_squared_error(t2_y,pred)*100
#95.85614903949983 without parameter
#95.36991383075062 on validation set with normal tuning

In [ ]:
train_x=train.drop(['amount_spent_per_room_night_scaled'],axis=1)
train_y=train['amount_spent_per_room_night_scaled']
test_x=test

In [ ]:
estimator = LGBMRegressor()

params = {
    'learning_rate': [0.03],
    'boosting_type':['gbdt'],
    'n_estimators': [630,650],
    'subsample':[0.75,0.8],
    'num_leaves':[50],
    'colsample_bytree':[0.5,0.57],
   'reg_alpha' : [5.5],
    'reg_lambda' : [4],
    'objective':['regression']
    }
lgbm = GridSearchCV(estimator,param_grid=params,cv=4,verbose=1,n_jobs=-1)


lgbm.fit(train_x,train_y)

In [ ]:
lgbm.best_estimator_

In [ ]:
pred=lgbm.predict(test)
pred=pd.DataFrame(pred)
pred.columns=['amount_spent_per_room_night_scaled']
final=pd.concat([reservid,pred],axis=1)
pd.DataFrame(final).to_csv("hack2.csv",index=False)